# Panel data analysis with Python

In [ ]:
import numpy as np
import pandas as pd

from linearmodels import PooledOLS          # Pooled model
from linearmodels import RandomEffects      # Random-effect model
from linearmodels import PanelOLS           # Fixed-effect model
from linearmodels import FirstDifferenceOLS # First difference model

from linearmodels.panel import compare      # Compare the results of multiple models
from statsmodels.api import add_constant    # for matrices of regression design

## Подготовка панельного датасета

Рассмотрим датасет `Gasoline`. Загрузим из файла как DataFrame

In [ ]:
df = pd.read_csv('../datasets/panels-plm/Gasoline.csv')
df.head()

Для задания панельной структуры необходимо задать мульти-индекс поверх обычного датафрейма

Используем метод `.set_index(['country', 'year'])` (Порядок индексов важен! Сначала индивидуальный, потом временной)

In [ ]:
panel_df = df.set_index(['country', 'year'])
panel_df.head()

## Подгонка модели через формулу

__Важно__ в формуле обязательно нужно указать `1+` для учёта константы: `endog~1+exog1+...`. Иначе будет оцениваться _модель без константы_

### Модель Пула (OLS-оценивание)

In [ ]:
mod_pl = PooledOLS.from_formula(formula='lgaspcar~1+lincomep+lrpmg+lcarpcap', data=panel_df)
res_pl = mod_pl.fit()
res_pl

### Модель Случайных эффектов/Random Effects/RE (GLS-оценивание)

In [ ]:
mod_re = RandomEffects.from_formula(formula='lgaspcar~1+lincomep+lrpmg+lcarpcap', data=panel_df)
res_re = mod_re.fit()
res_re

### Модель Фиксированных эффектов/Fixed Effects/FE (Within-оценка)

- нужно указать `drop_absorbed=True` для автоматического удаление постоянных во времени переменных (иначе будет сообщение об ошибке в случае наличия таковых)
- в спецификацию нужно добавить в конце `+EntityEffects` для учёта индивидуальных эффектов и `+TimeEffects` для учёта временных эффектов (иначе будут OLS-оценки коэффициентов)

In [ ]:
mod_fe = PanelOLS.from_formula(formula='lgaspcar~1+lincomep+lrpmg+lcarpcap+EntityEffects', data=panel_df, drop_absorbed=True)
res_fe = mod_fe.fit()
res_fe

### Модель Фиксированных эффектов/Fixed Effects/FE (FD-оценка)

__Важно__ не включаем константу

In [ ]:
mod_fd = FirstDifferenceOLS.from_formula(formula='lgaspcar~lincomep+lrpmg+lcarpcap', data=panel_df)
res_fd = mod_fd.fit()
res_fd

## Панельные регрессии с робастными инференциями

Подгоним модели с оценкой ковариационной матрицы Ареллано-Бонда

Используем метод `.fit(cov_type='clustered', cluster_entity=True)`

Выведем результаты подгонки в одной таблице (t-статистик в скобках)

In [ ]:
mod_pl = PooledOLS.from_formula(formula='lgaspcar~1+lincomep+lrpmg+lcarpcap', data=panel_df)
mod_re = RandomEffects.from_formula(formula='lgaspcar~1+lincomep+lrpmg+lcarpcap', data=panel_df)
mod_fe = PanelOLS.from_formula(formula='lgaspcar~1+lincomep+lrpmg+lcarpcap+EntityEffects', data=panel_df)
mod_fd = FirstDifferenceOLS.from_formula(formula='lgaspcar~lincomep+lrpmg+lcarpcap', data=panel_df)

res_pl = mod_pl.fit(cov_type='clustered', cluster_entity=True)
res_re = mod_re.fit(cov_type='clustered', cluster_entity=True)
res_fe = mod_fe.fit(cov_type='clustered', cluster_entity=True)
res_fd = mod_fd.fit(cov_type='clustered', cluster_entity=True)

compare({'Pool': res_pl, 'RE': res_re, 'FE': res_fe, 'FD':res_fd}, stars=True)

Подгоним модели с оценкой ковариационной матрицы Дрисколла-Края

Используем метод `.fit(cov_type='kernel')` 

Выведем результаты подгонки в одной таблице (стандартные ошибки в скобках)

In [ ]:
mod_pl = PooledOLS.from_formula(formula='lgaspcar~1+lincomep+lrpmg+lcarpcap', data=panel_df)
mod_re = RandomEffects.from_formula(formula='lgaspcar~1+lincomep+lrpmg+lcarpcap', data=panel_df)
mod_fe = PanelOLS.from_formula(formula='lgaspcar~1+lincomep+lrpmg+lcarpcap+EntityEffects', data=panel_df)
mod_fd = FirstDifferenceOLS.from_formula(formula='lgaspcar~lincomep+lrpmg+lcarpcap', data=panel_df)

res_pl = mod_pl.fit(cov_type='kernel')
res_re = mod_re.fit(cov_type='kernel')
res_fe = mod_fe.fit(cov_type='kernel')
res_fd = mod_fd.fit(cov_type='kernel')

compare({'Pool': res_pl, 'RE': res_re, 'FE': res_fe, 'FD':res_fd}, stars=True, precision='std_errors')

## Подгонка моделей через матрицы регрессионного дизайна

__Замечание__ предварительно необходимо удалить наблюдения с пропущенными значениями

In [ ]:
dependent = ['lgaspcar']
regressors = ['lincomep', 'lrpmg', 'lcarpcap']

y = panel_df[dependent+regressors].dropna()[dependent]
X = add_constant( panel_df[dependent+regressors].dropna()[regressors] )
# For FD-estimator we do not include intercept
X_fd = panel_df[dependent+regressors].dropna()[regressors]

In [ ]:
mod_pl = PooledOLS(y, X)
mod_re = RandomEffects(y, X)
mod_fe = PanelOLS(y, X, entity_effects=True, drop_absorbed=True)
mod_fd = FirstDifferenceOLS(y, X_fd)

res_pl = mod_pl.fit()
res_re = mod_re.fit()
res_fe = mod_fe.fit()
res_fd = mod_fd.fit()

In [ ]:
compare({'Pool': res_pl, 'RE': res_re, 'FE': res_fe, 'FD':res_fd}, stars=True, precision='std_errors')